In [1]:
# plotting  
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

# data
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# modeling
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

# evaluation
from sklearn.metrics import r2_score
from sklearn.feature_selection import RFE
from statsmodels.stats.outliers_influence import variance_inflation_factor

import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('day.csv')

In [3]:
data.head()

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,01-01-2018,1,0,1,0,6,0,2,14.110847,18.18125,80.5833,10.749882,331,654,985
1,2,02-01-2018,1,0,1,0,0,0,2,14.902598,17.68695,69.6087,16.652113,131,670,801
2,3,03-01-2018,1,0,1,0,1,1,1,8.050924,9.47025,43.7273,16.636703,120,1229,1349
3,4,04-01-2018,1,0,1,0,2,1,1,8.200000,10.60610,59.0435,10.739832,108,1454,1562
4,5,05-01-2018,1,0,1,0,3,1,1,9.305237,11.46350,43.6957,12.522300,82,1518,1600


#### Reviewing data dictionary

Before we load in the data, let's review the data dictionary provided to get a casual understanding of what kind of data we will be working with. Once we have an understanding of each variable, we can further examine the data each variable associated with each variable.

#### Data dictionary
    - instant: record index
	- dteday : date
	- season : season (1:spring, 2:summer, 3:fall, 4:winter)
	- yr : year (0: 2018, 1:2019)
	- mnth : month ( 1 to 12)
	- holiday : weather day is a holiday or not (extracted from http://dchr.dc.gov/page/holiday-schedule)
	- weekday : day of the week
	- workingday : if day is neither weekend nor holiday is 1, otherwise is 0.
	+ weathersit : 
		- 1: Clear, Few clouds, Partly cloudy, Partly cloudy
		- 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
		- 3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
		- 4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog
	- temp : temperature in Celsius
	- atemp: feeling temperature in Celsius
	- hum: humidity
	- windspeed: wind speed
	- casual: count of casual users
	- registered: count of registered users
	- cnt: count of total rental bikes including both casual and registered

We can summarize a few things we've learned from reviewing the data dictionary:

1. We will have the opportunity to leverage temporal data like season, month, year and day. Temporal data is very useful when looking at problems that require a time series analysis. While we may not be sure just yet, what granularity of temporal data will be useful, we'll have to pay close attention to these features during individual variable analysis. 

undefined. Weather information like weather situation, temperature, humidity and windspeed, will hopefully allow us to predict demand by providing a glimpse into mind of the consumer. Will we see that weather conditions impact demand for bike sharing? These are great features to work with. It's important to not that weather situation has many categorical variables that are currently being associated to integer value of 1-4. These values should be treated as nominal and not ordinal data. We'll need to clean and pre-process ahead of modeling. Noting this now, so that we can re-visit this later.

undefined. While granular features like the ones describe above could potentially be helpful predictors, we also have data that represents the total count of casual and registered riders in the network. While this may not itself be a predictor of ride sharing demand, yet to be determined, they will ultimately help us find correlations. 

undefined. Last but not least we have the count of total rental bikes which is a sum of both registered and casual users defined above. This feature represents our target feature and the rest of the variables represent potential predictor variables.

#### Removing not needed data

We know from our look into the data dictionary that there are already a few variables that we don't need for our analysis. 

'instant' is a feature that represents the index of each data row. This is not needed once we bring our data into pandas.

'dteday'. Much of the data present in this feature is already represented in the mnth and yr column. As such, we can remove this in favor of leveraging those columns.

'casual' represents the count of casual users which is a subset of our target variable 'cnt'. Because we're interested in 'cnt', we can remove this feature.

Similar to 'casual', 'registered' represents the count of registered users. As this is a subset of our target variable 'cnt', we can remove this feature as we'd like to find correlations with our target variable and not a subset of that variable. 

In [4]:
data.drop('instant', inplace=True, axis=1)
data.drop('dteday', inplace=True, axis=1)
data.drop('casual', inplace=True, axis=1)
data.drop('registered', inplace=True, axis=1)

Let's take a quick look a the columns, null value count and data types of the features we have to work with.

In [5]:
data.info()
data.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 730 entries, 0 to 729
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   season      730 non-null    int64  
 1   yr          730 non-null    int64  
 2   mnth        730 non-null    int64  
 3   holiday     730 non-null    int64  
 4   weekday     730 non-null    int64  
 5   workingday  730 non-null    int64  
 6   weathersit  730 non-null    int64  
 7   temp        730 non-null    float64
 8   atemp       730 non-null    float64
 9   hum         730 non-null    float64
 10  windspeed   730 non-null    float64
 11  cnt         730 non-null    int64  
dtypes: float64(4), int64(8)
memory usage: 68.6 KB


season        0
yr            0
mnth          0
holiday       0
weekday       0
workingday    0
weathersit    0
temp          0
atemp         0
hum           0
windspeed     0
cnt           0
dtype: int64

We are very fortunate and it looks like there is no null values in our data set. However, we'll have to do a more granular investigation as sometimes values like 0.0 are just as uninformative as null values depending on how the data was prepared. 

## Data understanding

Many of the features in our dataset require transformation before we can visualize it. Let's go ahead and work through each potential predictor feature first, clean it and transform it into the appropriate data type and review some statistical information about its distribution.

Re-usable utility functions
Let us build some re-usable utility functions to help us understand our data. While these will be short functions, if we'd like to make changes later, it will be much easier to scale this across all variables.

In [6]:
def describe(data, column1):
    """
    Prints a set of summary statistics.

    :param p1: dataframe
    :param p2: column from dataframe containing values 
    """
    print("DESCRIPTIVE SUMMARY of " + str(column1))
    print(data[column1].describe())
    print("\n")
    print("UNIQUE VALUE COUNT: " + str(len(data[column1].unique())))

def plot_hist(data, column1):
    """
    Creates a plotly express histogram.

    :param p1: dataframe
    :param p2: column from dataframe containing values 
    """
    fig = px.histogram(data,x=column1, title="Histogram of values for " + str(column1))
    fig.show()


def plot_box(data, column1):
    """
    Creates a plotly express box plot leveraging a single column against target variables of interest.

    :param p1: dataframe
    :param p2: column from dataframe containing values 
    """
    fig_count = px.box(data,x=column1,y='cnt', title="Boxplot of values for " + str(column1) + " against our target variable cnt",)
    fig_count.show()


def create_feature_analysis(data, column1):
    """
    Creates wrapper function to return summary statistics and vizualization for a single column

    :param p1: dataframe
    :param p2: column from dataframe containing values 
    """
    describe(data, column1)
    plot_hist(data, column1)
    plot_box(data, column1)


def create_heatmap(data):
    # Generate correlation matrix
    data_corr = data.corr() 

    # Plot heatmap of correlation matrix
    fig = go.Figure()
    fig.add_trace(
        go.Heatmap(
            x=data_corr.columns,
            y=data_corr.columns,
            z=np.array(data_corr)
        )
    )

    fig.update_layout(
        height=800,
    )

    fig.show()

We notice that the unique values are presented as ordinal values. Because the values currently indicate there might be some sort of order of importance, let's change these variables into integers representing their string season. We'll need to do some post processing to create categorical dummy variables later, but this will be useful for visualization.

In [7]:
data.season = data.season.map({1:'spring', 2:'summer', 3: 'fall', 4: 'winter'})
data.mnth = data.mnth.map({1: "jan", 2:"feb", 3:"march", 4:"april", 5:"may", 6:"june", 7:"july", 8:"august", 9:"september", 10:"october", 11:"november", 12:"december"})
data.weekday = data.weekday.map({0:'monday',1:'tuesday',2:'wednesday',3:'thursday', 4:"friday", 5:"saturday", 6:'sunday'})
data.weathersit = data.weathersit.map({1:'clear',2:'misty',3:'light_snow',4:'heavy_rain_and_ice'})

In [8]:
create_feature_analysis(data, 'season')

DESCRIPTIVE SUMMARY of season
count      730
unique       4
top       fall
freq       188
Name: season, dtype: object


UNIQUE VALUE COUNT: 4


In [9]:
create_feature_analysis(data, 'season')

DESCRIPTIVE SUMMARY of season
count      730
unique       4
top       fall
freq       188
Name: season, dtype: object


UNIQUE VALUE COUNT: 4


In [10]:
create_feature_analysis(data, 'season')

DESCRIPTIVE SUMMARY of season
count      730
unique       4
top       fall
freq       188
Name: season, dtype: object


UNIQUE VALUE COUNT: 4


In [11]:
create_feature_analysis(data, 'yr')

DESCRIPTIVE SUMMARY of yr
count    730.000000
mean       0.500000
std        0.500343
min        0.000000
25%        0.000000
50%        0.500000
75%        1.000000
max        1.000000
Name: yr, dtype: float64


UNIQUE VALUE COUNT: 2


Our histogram indicates that our data is evenly distributed between year 0 and year 1. Our box plots tell an interesting story though. We can see that the number of casual and registered users went up in year 1. In addition, the median, max and min of registered users appears to jump significantly more than casual users.

In [12]:
create_feature_analysis(data, 'mnth')

DESCRIPTIVE SUMMARY of mnth
count     730
unique     12
top       jan
freq       62
Name: mnth, dtype: object


UNIQUE VALUE COUNT: 12


We can see that the median count of bike rentals goes up during the summer months and slowly starts to drop towards the end of the year. This could be helpful for us during the modeling phase when we're looking to identify features with high correlation.

In [13]:
create_feature_analysis(data, 'holiday')

DESCRIPTIVE SUMMARY of holiday
count    730.000000
mean       0.028767
std        0.167266
min        0.000000
25%        0.000000
50%        0.000000
75%        0.000000
max        1.000000
Name: holiday, dtype: float64


UNIQUE VALUE COUNT: 2


We can see that the distribution of riders increased during holidays. However the max number of riders out on the street remained relevailty the same.

In [14]:
create_feature_analysis(data, 'weekday')

DESCRIPTIVE SUMMARY of weekday
count        730
unique         7
top       sunday
freq         105
Name: weekday, dtype: object


UNIQUE VALUE COUNT: 7


Nothing out of the ordinary here. Data for each of day of the week is evenly distributed.

In [15]:
create_feature_analysis(data, 'workingday')

DESCRIPTIVE SUMMARY of workingday
count    730.000000
mean       0.683562
std        0.465405
min        0.000000
25%        0.000000
50%        1.000000
75%        1.000000
max        1.000000
Name: workingday, dtype: float64


UNIQUE VALUE COUNT: 2


Although the mean and max are relavitely the same, we can that there are consistently more riders out on the weekend.

In [16]:
create_feature_analysis(data, 'weathersit')

DESCRIPTIVE SUMMARY of weathersit
count       730
unique        3
top       clear
freq        463
Name: weathersit, dtype: object


UNIQUE VALUE COUNT: 3


Now we're getting somewhere. The distribution of weather situations indicates that the data has more clear entries than misty or light snow entries. It also highlights the fact that there are no heavy rain and ice entries despite the dataset indicates as such.

In [17]:
create_feature_analysis(data, 'temp')

DESCRIPTIVE SUMMARY of temp
count    730.000000
mean      20.319259
std        7.506729
min        2.424346
25%       13.811885
50%       20.465826
75%       26.880615
max       35.328347
Name: temp, dtype: float64


UNIQUE VALUE COUNT: 498


As the temperature increases, we can see that the distribution of total bike rentals varies significantly.

In [18]:
create_feature_analysis(data, 'atemp')

DESCRIPTIVE SUMMARY of atemp
count    730.000000
mean      23.726322
std        8.150308
min        3.953480
25%       16.889713
50%       24.368225
75%       30.445775
max       42.044800
Name: atemp, dtype: float64


UNIQUE VALUE COUNT: 689


In [19]:
create_feature_analysis(data, 'hum')

DESCRIPTIVE SUMMARY of hum
count    730.000000
mean      62.765175
std       14.237589
min        0.000000
25%       52.000000
50%       62.625000
75%       72.989575
max       97.250000
Name: hum, dtype: float64


UNIQUE VALUE COUNT: 594


In [20]:
create_feature_analysis(data, 'windspeed')

DESCRIPTIVE SUMMARY of windspeed
count    730.000000
mean      12.763620
std        5.195841
min        1.500244
25%        9.041650
50%       12.125325
75%       15.625589
max       34.000021
Name: windspeed, dtype: float64


UNIQUE VALUE COUNT: 649


In [21]:
create_feature_analysis(data, 'windspeed')

DESCRIPTIVE SUMMARY of windspeed
count    730.000000
mean      12.763620
std        5.195841
min        1.500244
25%        9.041650
50%       12.125325
75%       15.625589
max       34.000021
Name: windspeed, dtype: float64


UNIQUE VALUE COUNT: 649


## Data preperation

Before we go and explore our target variable, let's go ahead and re-transform our potential predictor features into numerical data types so that we can leverage a correlation matrices to tell us more about feature correlation.  

In [22]:
data.head()

,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,cnt
0,spring,0,jan,0,sunday,0,misty,14.110847,18.18125,80.5833,10.749882,985
1,spring,0,jan,0,monday,0,misty,14.902598,17.68695,69.6087,16.652113,801
2,spring,0,jan,0,tuesday,1,clear,8.050924,9.47025,43.7273,16.636703,1349
3,spring,0,jan,0,wednesday,1,clear,8.200000,10.60610,59.0435,10.739832,1562
4,spring,0,jan,0,thursday,1,clear,9.305237,11.46350,43.6957,12.522300,1600


In [23]:
# creating feature columns for each categorical feature in our dataset. 

mnths_dummies = pd.get_dummies(data.mnth, drop_first=True)
weekdays_dummies = pd.get_dummies(data.weekday,drop_first=True)
weathersit_dummies = pd.get_dummies(data.weathersit,drop_first=True)
seasons_dummies = pd.get_dummies(data.season,drop_first=True)

In [24]:
# adds those columns to a new dataset along with the rest of our data
numerical_data = pd.concat([data, mnths_dummies, weekdays_dummies, weathersit_dummies, seasons_dummies], axis=1)
# removes original variables from new data set.
numerical_data.drop(['mnth','weekday', 'weathersit', 'season'], inplace=True, axis=1)

In [25]:
numerical_data.head()

,yr,holiday,workingday,temp,atemp,hum,windspeed,cnt,august,december,...,saturday,sunday,thursday,tuesday,wednesday,light_snow,misty,spring,summer,winter
0,0,0,0,14.110847,18.18125,80.5833,10.749882,985,0,0,...,0,1,0,0,0,0,1,1,0,0
1,0,0,0,14.902598,17.68695,69.6087,16.652113,801,0,0,...,0,0,0,0,0,0,1,1,0,0
2,0,0,1,8.050924,9.47025,43.7273,16.636703,1349,0,0,...,0,0,0,1,0,0,0,1,0,0
3,0,0,1,8.200000,10.60610,59.0435,10.739832,1562,0,0,...,0,0,0,0,1,0,0,1,0,0
4,0,0,1,9.305237,11.46350,43.6957,12.522300,1600,0,0,...,0,0,1,0,0,0,0,1,0,0


Now that we have a better understanding of our potential predictor variables and they'e been converted to numerical variables (either continuous or categorical), let's go ahead gain a better understanding of our target variable and do any cleaning, transformation needed. 

In [26]:
create_feature_analysis(data, 'cnt')

DESCRIPTIVE SUMMARY of cnt
count     730.000000
mean     4508.006849
std      1936.011647
min        22.000000
25%      3169.750000
50%      4548.500000
75%      5966.000000
max      8714.000000
Name: cnt, dtype: float64


UNIQUE VALUE COUNT: 695


Our target variable looks like its in good condition. Let's visualize a quick heat mat to see if we can find any obvious correlations now that all of our data is numerical.

In [27]:
create_heatmap(numerical_data)

It appears that there is some positive correlation between certain variables and some negative correlation with others. However finding a correlation between to potential predictor variables isn't necessarily helpful. We will evaluate this more when we find VIF scores shortly.

We do see some indication that certain variables like windspeed, workingday, holiday are strong negative factors and yr, atemp are positive factors. Let's see what a quick scatter matrix shows us about the positive correlations we potentially found.

## Dataset building

First, let's split the data into test and train datasets. 

In [28]:
# seprating our target and predictor variables
data_train, data_test = train_test_split(numerical_data, train_size = 0.7, test_size = 0.3, random_state = 100)

In [29]:
data_test.head()

,yr,holiday,workingday,temp,atemp,hum,windspeed,cnt,august,december,...,saturday,sunday,thursday,tuesday,wednesday,light_snow,misty,spring,summer,winter
184,0,1,0,29.793347,33.27085,63.7917,5.459106,6043,0,0,...,0,0,0,1,0,0,1,0,0,0
535,1,0,1,32.082500,36.04875,59.2083,7.625404,6211,0,0,...,0,0,1,0,0,0,0,0,1,0
299,0,0,1,19.270000,22.85230,81.2917,13.250121,2659,0,0,...,0,0,0,0,0,0,1,0,0,1
221,0,0,1,31.433347,34.24915,42.4167,13.417286,4780,1,0,...,0,0,1,0,0,0,0,0,0,0
152,0,0,1,29.315000,32.19710,30.5000,19.583229,4968,0,0,...,0,0,0,0,0,0,0,0,1,0


To aid in modeling, let's go ahead and re-scale our test and training numerical non categorical features using a standard scalar which rescales variables so that there is a mean of 0. Noting that we are _only_ fitting and transforming on the training on the train set and transforming on the test set.

In [30]:
scaler = StandardScaler()
continuous_features = ['temp','atemp','hum','windspeed','cnt']
data_train[continuous_features] = scaler.fit_transform(data_train[continuous_features])
data_test[continuous_features] = scaler.transform(data_test[continuous_features])

Let's extract our target and predictor variables in the training set for modeling.

In [31]:
y_train = data_train.pop('cnt')
X_train = data_train

## Modeling

### Linear regression model variation 1

In [33]:
lm_variation_1 = LinearRegression()
lm_variation_1.fit(X_train, y_train)

LinearRegression()

Let's get predictions and calculate the r^2 value.

In [34]:
y_pred = lm_variation_1.predict(X_train)
r2_score(y_pred, y_train)

0.8273373177762576

While this score isn't bad, let's see if we can do better. Let's leverage recursive feature elimination for feature selection.

### Linear regression model variation 2 with RFE

First we have to fit an RFE estimator and let it know what model to use and how many feature to select.

In [35]:
rfe_lm_variation_2 = RFE(estimator=LinearRegression(), n_features_to_select=15)   
rfe = rfe_lm_variation_2.fit(X_train, y_train)

Let's print out the features in our dataset and the score the RFE model ranks them. A score of 0 is a good value whereas scores that are large are ones that we should consider removing. 

In [36]:
list(zip(X_train.columns,rfe.support_,rfe.ranking_))

[('yr', True, 1),
 ('holiday', True, 1),
 ('workingday', True, 1),
 ('temp', True, 1),
 ('atemp', False, 10),
 ('hum', False, 6),
 ('windspeed', False, 4),
 ('august', False, 15),
 ('december', True, 1),
 ('feb', False, 2),
 ('jan', True, 1),
 ('july', True, 1),
 ('june', False, 9),
 ('march', False, 14),
 ('may', False, 5),
 ('november', True, 1),
 ('october', False, 12),
 ('september', True, 1),
 ('monday', True, 1),
 ('saturday', False, 13),
 ('sunday', True, 1),
 ('thursday', False, 11),
 ('tuesday', False, 7),
 ('wednesday', False, 8),
 ('light_snow', True, 1),
 ('misty', True, 1),
 ('spring', True, 1),
 ('summer', False, 3),
 ('winter', True, 1)]

Let's get only the columns that have a RFE support of True.

In [37]:
col = X_train.columns[rfe.support_]
col

Index(['yr', 'holiday', 'workingday', 'temp', 'december', 'jan', 'july',
       'november', 'september', 'monday', 'sunday', 'light_snow', 'misty',
       'spring', 'winter'],
      dtype='object')

Now we can create a new data frame by filtering out non useful columns

In [38]:
X_train_rfe = X_train[col]

### Attaining VIF scores

VIF scores are important because they help us understand if there are more than 1 variable that have a strong correlation with one another. If another variable 'explains away' other variables, we can reduce the number of variables in our dataset without impacting our score. A common herustic for VIF scores is that we'd like to get them as low as possible. Below 5 is a good heuristic going in.

In [39]:
# creating an empty dataframe for our vif scores.
def create_vif_scores(data):
    
    vif = pd.DataFrame()
    vif['features'] = data.columns
    vif['vif'] = [variance_inflation_factor(data.values, i) for i in range(data.shape[1])]
    vif['vif'] = round(vif['vif'], 2)
    vif = vif.sort_values(by = "vif", ascending = False)
    return vif

In [40]:
create_vif_scores(X_train_rfe)

,features,vif
2,workingday,3.87
3,temp,3.19
13,spring,2.74
14,winter,2.27
7,november,1.70
10,sunday,1.66
9,monday,1.60
5,jan,1.59
4,december,1.42
6,july,1.30


It's generally accepted that a VIF score below 5 is good. However, let's create a model using stats model library where we're able to look at more information about each features impact to our model performance.

### Linear regression model variation 3

In [41]:
X_train_rfe_variation_1 = sm.add_constant(X_train_rfe)
lm_variation_3 = sm.OLS(y_train,X_train_rfe_variation_1).fit()   # Running the linear model
print(lm_variation_3.summary())

                            OLS Regression Results                            
Dep. Variable:                    cnt   R-squared:                       0.832
Model:                            OLS   Adj. R-squared:                  0.828
Method:                 Least Squares   F-statistic:                     175.4
Date:                Tue, 20 Sep 2022   Prob (F-statistic):          1.47e-181
Time:                        22:12:06   Log-Likelihood:                -268.41
No. Observations:                 510   AIC:                             566.8
Df Residuals:                     495   BIC:                             630.3
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.3211      0.041     -7.753      0.0

Looking at the above stats model, we can see that 't' scores are generall in a good place. < -2 or > 2. We also see that the majority of P scores are near 0, which is great. Our model currently has 83% explainability of our data and our F-statistic score is high. Great news! Let's see if we can make an improvement to the model but eliminating features that have non 0 p scores.

In [42]:
col = col.drop(['december'])

In [43]:
create_vif_scores(X_train_rfe[col])

,features,vif
2,workingday,3.80
3,temp,2.99
12,spring,2.74
13,winter,2.00
9,sunday,1.64
8,monday,1.59
4,jan,1.52
6,november,1.46
5,july,1.29
1,holiday,1.16


### Linear regression model variation 4

Now that we've reduced the VIF scores all around, let's train another stats model on the new columns. As we're nearing the end of our modeling, lets reassign the new columns selected back to our X_train variable. 

In [45]:
X_train_rfe = X_train_rfe[col]

In [46]:
X_train_rfe_with_constant = sm.add_constant(X_train_rfe)
lm_variation_4 = sm.OLS(y_train,X_train_rfe_with_constant).fit()  
print(lm_variation_4.summary())

                            OLS Regression Results                            
Dep. Variable:                    cnt   R-squared:                       0.830
Model:                            OLS   Adj. R-squared:                  0.826
Method:                 Least Squares   F-statistic:                     186.6
Date:                Tue, 20 Sep 2022   Prob (F-statistic):          1.88e-181
Time:                        22:13:31   Log-Likelihood:                -271.39
No. Observations:                 510   AIC:                             570.8
Df Residuals:                     496   BIC:                             630.1
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.3333      0.041     -8.066      0.0

## Evaluation

### Training set evaluation

In [47]:
y_train_pred = lm_variation_4.predict(X_train_rfe_with_constant)
train_r2 = r2_score(y_train, y_train_pred)
train_r2

0.8302778318345925

It looks like we increase our R2 score on the training set by .01. Great news!

### Residual analysis

One of the important analysis to do when evaluating linear models, is to learn if the error values are normally distributed. Let's plot the distribution of error values comparing y_pred and y_train.

In [48]:
train_errors = y_train - y_train_pred
px.histogram(train_errors, nbins=20, title="Errors")

We can see that the error values are distributed evenly and the mean hovers around 0. Before we evaluate our model with RMSE, let's identify whether or now we see any patterns in the residuals plotting against y_train. 

In [49]:
figure_1 = px.scatter(y_train, y_train, train_errors)
figure_1.add_hline(y=0, line_width=2, line_color="red")
figure_1.show()

There is no noticeable pattern between the training data and the errors.

In [50]:
figure_2 = px.scatter(y_train, y_train, y_train_pred)
figure_2.update_xaxes(title_text='Y train')
figure_2.update_yaxes(title_text='Y train pred')
figure_2.show()

We do see that y_train and y_pred are linearly correlated. 

### Test set evaluation

#### R^2 and adjusted R^2 evaluation on test data

In [51]:
def calculate_adj_r2(data, r2):
    adusted_r_2 = 1-(1-r2)*(len(data)-1)/(len(data)-len(data.columns)-1)
    return adusted_r_2

In [52]:
data_test.head()

,yr,holiday,workingday,temp,atemp,hum,windspeed,cnt,august,december,...,saturday,sunday,thursday,tuesday,wednesday,light_snow,misty,spring,summer,winter
184,0,1,0,1.305371,1.209703,0.047995,-1.394498,0.798166,0,0,...,0,0,0,1,0,0,1,0,0,0
535,1,0,1,1.613721,1.553413,-0.276087,-0.984730,0.884309,0,0,...,0,0,1,0,0,0,0,0,1,0
299,0,0,1,-0.112129,-0.079386,1.285378,0.079219,-0.937002,0,0,...,0,0,0,0,0,0,1,0,0,1
221,0,0,1,1.526280,1.330748,-1.463381,0.110839,0.150554,1,0,...,0,0,1,0,0,0,0,0,0,0
152,0,0,1,1.240938,1.076847,-2.305983,1.277165,0.246953,0,0,...,0,0,0,0,0,0,0,0,1,0


In [54]:
# creating test predictor feature dataframe and target feature frame
y_test = data_test.pop('cnt')
X_test = data_test

KeyError: 'cnt'

In [56]:
# ensuring that we're testing only on the columns that our model is set to predict on.
X_test = data_test[X_train.columns]
X_test.columns

Index(['yr', 'holiday', 'workingday', 'temp', 'jan', 'july', 'november',
       'september', 'monday', 'sunday', 'light_snow', 'misty', 'spring',
       'winter'],
      dtype='object')

In [58]:
# adding constant to X_train
X_test_rfe_with_constant = sm.add_constant(X_test)

y_test_pred = lm_variation_4.predict(X_test_rfe_with_constant)
test_data_r2 = r2_score(y_test, y_test_pred)
test_data_r2

0.8154914578483347

Not bad! While .06 lower than our training set, this indicates that our model did not memorize that data and can perform nearly as well on new data.

Because we're doing a multivariate linear regression, let's also measure adjusted r^2 score.

In [59]:
print("The adjusted r_2 score for our test predictions is ", round(calculate_adj_r2(X_test, test_data_r2), 5))

The adjusted r_2 score for our test predictions is  0.80283


Our model is performing as expected and desired. Let's take a look at the coefficients of the model and map them back to our features so that we can evaluate feature importance.

In [64]:
px.bar(round(lm_variation_4.params,5))

## Summary

#### Top 5 predictors of bike demand.

As a recommendation to our business partners looking to predict bike rental demand, we'd recommend leveraging the following features when productionizing a linear model. 

1. <b>light_snow</b>
    If there is light snow, its likely that we'll see a decrease in the demand for bike rentals.
2. <b>yr</b>
    The year in which an individiaul decies to register for a bike share has incredible impact as a predictor of demand. It is our largest predictor with a positive correlation to demand count. As ride sharing company continues to grow, it makes sense that demand would increase as well as popularity and word of mouth increases. 
3. <b>temp</b>
    Temperature is our second largest predictor with a positive crrelation. As the temperature rises, we can see a slow increase in the demand for bike rentals. 
4. <b>spring</b>
    Interestingly enough, during spring there is a slight negative correlation with increase bike rental demand. 
5. <b>winter</b>
    Last but not least, there is slightly higher demand for bike rentals during the winter, but lower demand if its snowing!

As a final confirmation of our recommendations above, let's look at the features above (that are categorical) and their impact to our target variable 'cnt''s mean.

In [58]:
numerical_data.groupby(['light_snow'])['cnt'].mean()

light_snow
0    4588.118477
1    1803.285714
Name: cnt, dtype: float64

In [59]:
numerical_data.groupby(['yr'])['cnt'].mean()

yr
0    3405.761644
1    5610.252055
Name: cnt, dtype: float64

In [60]:
numerical_data.groupby(['spring'])['cnt'].mean()

spring
0    5129.692727
1    2608.411111
Name: cnt, dtype: float64

In [61]:
numerical_data.groupby(['winter'])['cnt'].mean()

winter
0    4437.014493
1    4728.162921
Name: cnt, dtype: float64

As expected each of the categorical variables above do make an impact the mean of our target variable. It makes sense that these are the most impactful predictor variables.

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=e081c118-f72c-47d9-bcd6-0893c59319a8' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>